## import and load

In [6]:
from utils import *

In [7]:
!pwd

/Users/roccc/Desktop/Projects/jpnb/[202304]谣言可解释


In [13]:
pheme_path = '/Users/roccc/Desktop/Projects/rumor_detection/data/pheme-2/PHEME/6392078/PHEME_veracity/all-rnr-annotated-threads'
set_workspace(pheme_path)

workspace: /Users/roccc/Desktop/Projects/rumor_detection/data/pheme-2/PHEME/6392078/PHEME_veracity/all-rnr-annotated-threads


In [19]:
def load_json(filepath, encoding='utf-8'):
    with open(filepath, 'r', encoding=encoding) as fh:
        return json.load(fh)

## extract features

Textual features:
1. length
2. sentiment,
3. topic, 
4. has_image/media,
    - `attachments.media_keys` or `media`
5. has_hashtag,
6. has_mention,
7. has_url,

User features:
1. \# followers,
2. \# followees,
3. \# posts,
<!-- 4. gender, -->
5. location,
6. verified,
7. geo_enabled

Propagation features:
1. \# favorates,
2. \# retweets,
3. \# comments,
4. size,
5. depth,

- length, senti, topic 用events_text.json抽；
- has_image, hashtag, mention, url, 用原数据中0节点的json抽；
- user part, gender, location用原数据中的0节点的json的user抽；
- propagation用已经抽好的pheme pyg dataset抽；num_nodes, num_edges, propatation_depth

## 用原始数据抽

In [20]:
def get_features_by_node(node):
    nid = node['id']
    has_image = len(node.get('attachments',[]))
    has_hashtag = len(node.get('entities').get('hashtags',[]))
    has_mention = len(node.get('entities').get('user_mentions',[]))
    has_url = len(node.get('entities').get('urls',[]))
    
    # user features
    location = node.get('user').get('location')
    verified = node.get('user').get('verified')
    geo_enabled = node.get('user').get('geo_enabled')
    
    return [has_image,has_hashtag,has_mention, has_url, location, verified, geo_enabled]

# def get_user_features_by_node(node):
#     nid = node['id']
# #     gender = node.get('user',[])
#     location = node.get('user').get('location')
#     verified = node.get('user').get('verified')
#     geo_enabled = node.get('user').get('geo_enabled')

#     return [location, verified, geo_enabled]

def get_node_features_by_event(eid, eid_path):
#   print(eid)
    event_nodes_features = {}
    
    source_node = load_json(f'{eid_path}/source-tweets/{eid}.json',encoding='windows-1252')
    event_nodes_features.update({eid: get_features_by_node(source_node)})
    return event_nodes_features

In [27]:
events_nodes_features_dict = {}
for folder in os.listdir(pheme_path):
    print(f'{folder}')
    if folder.endswith('threads'):
        file_path = os.path.join(pheme_path, folder)
    else:
        continue
    rumors_path = os.path.join(file_path, 'rumours')
    non_rumors_path = os.path.join(file_path, 'non-rumours')
    
    rumors_files = list(filter(lambda x: x[-1] != 'e', os.listdir(rumors_path)))
    non_rumors_files = list(filter(lambda x: x[-1] != 'e', os.listdir(non_rumors_path)))
    
    events_rumors_nodes_text = dict(map(lambda eid:
                                                (eid, get_node_features_by_event(eid, os.path.join(rumors_path,eid))),
                                               rumors_files))
    events_nodes_features_dict.update(events_rumors_nodes_text)

    
    events_non_rumors_nodes_text = dict(map(lambda eid:
                                                (eid, get_node_features_by_event(eid, os.path.join(non_rumors_path,eid))),
                                               non_rumors_files))
    events_nodes_features_dict.update(events_non_rumors_nodes_text)


.DS_Store
germanwings-crash-all-rnr-threads
prince-toronto-all-rnr-threads
ebola-essien-all-rnr-threads
ferguson-all-rnr-threads
charliehebdo-all-rnr-threads
putinmissing-all-rnr-threads
ottawashooting-all-rnr-threads
sydneysiege-all-rnr-threads
gurlitt-all-rnr-threads


In [28]:
# events_nodes_features_dict
events_nodes_features_dict = {id_: v[id_] for id_,v in events_nodes_features_dict.items()}

In [32]:
# len(events_nodes_features_dict)
save_as_json(events_nodes_features_dict, 'events_root_nodes_features_dict.json')

### text features

### user features

### propagation features